In [ ]:
import pandas as pd
import sqlite3 as s3
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn import metrics
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')

go_sales = s3.connect('go_sales.sqlite')

order_details = pd.read_sql_query('SELECT * FROM order_details', go_sales)
order_header = pd.read_sql_query('SELECT * FROM order_header', go_sales)
returned_item = pd.read_sql_query('SELECT * FROM returned_item', go_sales)

In [ ]:
merged_orders = pd.merge(order_details, order_header, left_on='ORDER_NUMBER', how='inner', right_on='ORDER_NUMBER')
returned_orders = pd.merge(merged_orders, returned_item, left_on='ORDER_DETAIL_CODE', how='outer', right_on='ORDER_DETAIL_CODE')
returned_orders = returned_orders[['PRODUCT_NUMBER', 'QUANTITY', 'UNIT_PRICE', 'UNIT_SALE_PRICE', 'RETAILER_SITE_CODE', 'ORDER_DATE', 'RETURN_DATE', 'RETURN_REASON_CODE']]
# returned_orders['RETURN_DATE'] = returned_orders['RETURN_DATE'].fillna(0)
# returned_orders['RETURN_REASON_CODE'] = returned_orders['RETURN_REASON_CODE'].fillna(0)
# returned_orders = returned_orders[returned_orders['RETURN_REASON_CODE'] != 0]
returned_orders['ORDER_DATE'] = pd.to_datetime(returned_orders['ORDER_DATE'])
returned_orders['RETURN_DATE'] = pd.to_datetime(returned_orders['RETURN_DATE'], format="%d-%m-%Y %H:%M:%S")
returned_orders['RETURN_DAY'] = returned_orders['RETURN_DATE'].dt.day
returned_orders['RETURN_MONTH'] = returned_orders['RETURN_DATE'].dt.month
returned_orders['RETURN_YEAR'] = returned_orders['RETURN_DATE'].dt.year
returned_orders = returned_orders.drop('RETURN_DATE', axis=1)
returned_orders['ORDER_DATE'] = pd.to_datetime(returned_orders['ORDER_DATE'])
returned_orders['ORDER_DAY'] = returned_orders['ORDER_DATE'].dt.day
returned_orders['ORDER_MONTH'] = returned_orders['ORDER_DATE'].dt.month
returned_orders['ORDER_YEAR'] = returned_orders['ORDER_DATE'].dt.year
returned_orders = returned_orders.drop('ORDER_DATE', axis=1)
# returned_orders = returned_orders[returned_orders['RETURN_REASON_CODE'].notna()]
df = returned_orders
df

In [ ]:
x = df.drop('RETURN_REASON_CODE', axis = 1)
y = df.loc[:, ['RETURN_REASON_CODE']]
x

x_train, x_text, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=42)

dtree = DecisionTreeClassifier(max_depth=2)
dtree = dtree.fit(x_train, y_train)
tree.plot_tree(dtree, feature_names=x.columns)
plt.show()